In [15]:
# Follow tutorial from here: https://huggingface.co/docs/transformers/v4.17.0/en/tasks/language_modeling
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
import json
import datasets
from transformers import AutoTokenizer, DataCollatorWithPadding, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling

# hyperparams
checkpoint = "gpt2-medium"

# set up dset
df = pd.read_json(open('titles_prepared.jsonl', 'r'), lines=True).astype(str)
df = df.sample(n=500)
df['text'] = df['prompt'] + df['completion']
df = df.drop(columns=['prompt', 'completion'])
dset = datasets.Dataset.from_pandas(df)

# set up tokenized data
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokenizer.pad_token = tokenizer.eos_token
def tokenize_function(ex):
    return tokenizer(ex["text"], padding='max_length', truncation=True)
dset_tokenized = dset.map(tokenize_function, batched=True)
dset_tokenized = dset_tokenized.add_column('labels', dset_tokenized['input_ids'])
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/gpt2-medium/resolve/main/config.json from cache at /home/chansingh/.cache/huggingface/transformers/3a7a4b7235202f93d14a4a5e8200709184c5b25a29d9cfa6b0ede5166adf0768.cf0ec4a33a38dc96108560e01338af4bd3360dd859385d451c35b41987ae73ff
Model config GPT2Config {
  "_name_or_path": "gpt2-medium",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 1024,
  "n_head": 16,
  "n_inner": null,
  "n_layer": 24,
  "n_positions": 1024,
  "n_special": 0,
  "predict_special_tokens": true,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_ac

  0%|          | 0/1 [00:00<?, ?ba/s]

In [3]:
model = AutoModelForCausalLM.from_pretrained(checkpoint)

In [13]:
training_args = TrainingArguments(
    output_dir=".",
    per_device_train_batch_size=1,
    num_train_epochs=1,
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dset_tokenized,
    data_collator=data_collator,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [14]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: text. If text are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.
/home/chansingh/.autoprompt/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 5465
  Num Epochs = 3
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 16395


Step,Training Loss


KeyboardInterrupt: 